In [5]:
import torch
from get_data_fs import *
from NN_classes_fs import *
from dataloader_fs import *
import os 
from tqdm import tqdm
import logging
from test_func_fs import *
from load_data import *

torch.set_default_dtype(torch.float64)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
%matplotlib inline

In [ ]:
x=torch.linspace(-5,5,1000)
y=torch.sigmoid(x)
y2=torch.tanh(x)
y3=torch.relu(x)


SMALL_SIZE = 15
MEDIUM_SIZE = 25
BIGGER_SIZE = 12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsiz

params = {'axes.titlesize': MEDIUM_SIZE}
plt.rcParams.update(params)

figure , axs = plt.subplots(1,3, figsize=(19,6))
figure.tight_layout(pad=2.0)


axs[0].plot(x,y3, linewidth=3)
axs[1].plot(x,y, linewidth=3)
axs[2].plot(x,y2, linewidth=3)

axs[0].set_title("ReLU")
axs[1].set_title("Sigmoid")
axs[2].set_title("Tanh")

for i in range(3):
    axs[i].grid()

In [ ]:
if os.name == "nt":
    print("true")
else:
    print(os.name)

In [ ]:
p1 = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_IV_sprung.csv"
p2 = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_IV2.csv"
p3 = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_same_u_und_mixed150.csv"
p4 = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_randomwalk_stationary_mix.csv"
data = get_data(path=p4, num_inits=10)
# % matplotlib qt
for i in range(5):
    visualise(data, i)

In [ ]:
# train function

def train_tcn(input_data, model,  optimizer, lr_scheduler, learning_rate=0.001):
 
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
 
    model.train()
    total_loss = []
  
    for k, (x,y) in enumerate(input_data):  # inp = (u, x) label = x
        
        x = x.to(device)
        y = y.to(device)

        x = x.transpose(1,2)
        y = y.transpose(1,2)

        out = model(x)
  
        # reset the gradient
        optimizer.zero_grad(set_to_none=True)
        
        # calculate the error
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
 
        total_loss.append(loss.detach().cpu().numpy())

    lr_scheduler.step()
 
   # return the average error of the next step prediction
    return np.mean(total_loss)

def train_lstm(input_data, model,  optimizer, lr_scheduler, learning_rate=0.001):
 
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
 
    model.train()
    total_loss = []
  
    for k, (x,y) in enumerate(input_data):  # inp = (u, x) label = x
        
        x = x.to(device)
        y = y.to(device)
        
        output, _ = model(x)
  
        # reset the gradient
        optimizer.zero_grad(set_to_none=True)
        
        # calculate the error
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
        total_loss.append(loss.detach().cpu().numpy())
        lr_scheduler.step()

    return np.mean(total_loss)

def train_mlp(loader, model,  optimizer, lr_scheduler, learning_rate=0.001):
 
    loss_fn = nn.MSELoss()

 
    model.train()
    total_loss = []
  
    for x,y in loader:  # inp = (u, x) label = x
        
        x = x.to(device)
        y = y.to(device)
 
        out = model(x)
 
        #print(output.size())
        # reset the gradient
        optimizer.zero_grad(set_to_none=True)
        
        # calculate the error
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
        total_loss.append(loss.detach().cpu().numpy())
         
    # before_lr = optimizer.param_groups[0]["lr"]
    lr_scheduler.step()
    # after_lr = optimizer.param_groups[0]["lr"]
    # print("SGD lr %.4f -> %.4f" % (before_lr, after_lr))
    

   # return the average error of the next step prediction
    return np.mean(total_loss)

In [ ]:
### mlp ###
params_mlp =    {
                        "window_size" : 20,
                        "h_size" : 24,
                        "l_num" : 3,
                        "learning_rate" : 0.001,
                        "batch_size" : 3,
                        "act_fn" : "relu",
                        "nonlin_at_out" : None, #None if no nonlinearity at the end

                        "percentage_of_data" : 0.9,
                        "cut_off_timesteps" : 0,
                        "part_of_data" : 1,
                        "epochs" : 100,
                        "test_every_epochs" : 3,
                        "input_channels" : 5,
                        "output" : 3
                    }

# Configure logging
log_file = 'training_fullsystem.log'
filemode = 'a' if os.path.exists(log_file) else 'w'
logging.basicConfig(filename=log_file, filemode=filemode, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize the mlp model
model_mlp = OR_MLP(input_size=5*params_mlp["window_size"], hidden_size=params_mlp["h_size"], l_num=params_mlp["l_num"], output_size=3,
                    act_fn=params_mlp["act_fn"], act_at_end = None, timesteps=params_mlp["window_size"]).to(device)

# Generate input data (the data is normalized and some timesteps are cut off)
input_data1 = get_data(path = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_IV_sprung.csv", num_inits=params_mlp["part_of_data"])
input_data2 = get_data(path = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_IV2.csv", num_inits=params_mlp["part_of_data"])
input_data3 = get_data(path = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_randomwalk.csv", num_inits=params_mlp["part_of_data"])
input_data4 = get_data(path = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_same_u_und_mixed150.csv", num_inits=params_mlp["part_of_data"])
input_data = torch.cat((input_data1, input_data2, input_data3, input_data4))
print(input_data.size())

#Split data into train and test sets
np.random.seed(1234)
num_of_inits_train = int(len(input_data)*params_mlp["percentage_of_data"])
train_inits = np.random.choice(np.arange(len(input_data)),num_of_inits_train,replace=False)
test_inits = np.array([x for x in range(len(input_data)) if x not in train_inits])
np.random.shuffle(train_inits)
np.random.shuffle(test_inits)
train_data = input_data[train_inits,:input_data.size(dim=1)-params_mlp["cut_off_timesteps"],:]
test_data = input_data[test_inits,:,:]

# dataloader for batching during training
train_set_mlp = custom_simple_dataset(train_data, window_size=params_mlp["window_size"])
train_loader_mlp = DataLoader(train_set_mlp, batch_size=params_mlp["batch_size"], pin_memory=True)

average_traj_err_train_mlp=[]
#Training loop

optimizer = torch.optim.AdamW(model_mlp.parameters(), lr = params_mlp["learning_rate"])
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 10, eta_min=0, last_epoch=-1, verbose='deprecated')

for e in tqdm(range(params_mlp["epochs"])):
    
    train_error = train_mlp(train_loader_mlp, model_mlp, optimizer=optimizer, lr_scheduler=lr_scheduler, learning_rate=params_mlp["learning_rate"])
    if e:
        print("Training error : ", train_error)

    # Every few epochs get the error MSE of the true data
    # compared to the network prediction starting from some initial conditions
    if (e+1)%params_mlp["test_every_epochs"] == 0:
        err_test_mlp = test(test_data.to(device), model_mlp, model_type="mlp", window_size=params_mlp["window_size"], display_plots=False, numb_of_inits = 1)
        average_traj_err_train_mlp.append(err_test_mlp)
        print(f"Average error over full trajectories: training data mlp: {err_test_mlp}")
        
# Save trained model
path_mlp = f'or_test_mlp.pth'

torch.save(model_mlp.state_dict(), path_mlp)

print(f"Run finished!")

# Log parameters

logging.info(f"hyperparams mlp: {params_mlp}")
logging.info(f"mlp {average_traj_err_train_mlp}")   
logging.info("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
logging.info("\n")



In [ ]:
### LSTM ###
params_lstm =    {
                        "window_size" : 16,
                        "h_size" : 8,
                        "l_num" : 3,
                        "learning_rate" : 0.0008,
                        "batch_size" : 20,
                        "percentage_of_data" : 0.8,
                        "cut_off_timesteps" : 100,
                        "part_of_data" : 50,
                        "epochs" : 200,
                        "test_every_epochs" : 20,
                        "input_channels" : 5,
                        "output" : 3
                    }

# Configure logging
log_file = 'training_fullsystem.log'
filemode = 'a' if os.path.exists(log_file) else 'w'
logging.basicConfig(filename=log_file, filemode=filemode, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize the LSTM model
model_lstm = OR_LSTM(input_size=5, hidden_size=params_lstm["h_size"], out_size=3, layers=params_lstm["l_num"], window_size=params_lstm["window_size"]).to(device)

# Generate input data (the data is normalized and some timesteps are cut off)
input_data1 = get_data(path = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_IV_sprung.csv", num_inits=params_lstm["part_of_data"])
input_data2 = get_data(path = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_IV2.csv", num_inits=params_lstm["part_of_data"])
input_data4 = get_data(path = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_01_same_u_und_mixed150.csv", num_inits=params_mlp["part_of_data"])
input_data = torch.cat((input_data1, input_data2, input_data3, input_data4))

print(input_data.size())

#Split data into train and test sets
np.random.seed(1234)
num_of_inits_train = int(len(input_data)*params_lstm["percentage_of_data"])
train_inits = np.random.choice(np.arange(len(input_data)),num_of_inits_train,replace=False)
test_inits = np.array([x for x in range(len(input_data)) if x not in train_inits])
np.random.shuffle(train_inits)
np.random.shuffle(test_inits)
train_data = input_data[train_inits,:input_data.size(dim=1)-params_lstm["cut_off_timesteps"],:]
test_data = input_data[test_inits,:,:]

# dataloader for batching during training
train_set_lstm = custom_simple_dataset(train_data, window_size=params_lstm["window_size"])
train_loader_lstm = DataLoader(train_set_lstm, batch_size=params_lstm["batch_size"], pin_memory=True)

average_traj_err_train_lstm=[]
#Training loop
for e in tqdm(range(params_lstm["epochs"])):
    
    train_error = train_lstm(train_loader_lstm, model_lstm, learning_rate= params_lstm["learning_rate"])
    if e % 10:
        print("Training error : ", train_error)

    # Every few epochs get the error MSE of the true data
    # compared to the network prediction starting from some initial conditions
    if (e+1)%params_lstm["test_every_epochs"] == 0:
        err_test_lstm = test(test_data.to(device), model_lstm, model_type="lstm", window_size=params_lstm["window_size"], display_plots=False, numb_of_inits = 1)
        average_traj_err_train_lstm.append(err_test_lstm)
        print(f"Average error over full trajectories: training data LSTM: {err_test_lstm}")
        
# Save trained model
path_lstm = f'Full_System/or_test_lstm.pth'

torch.save(model_lstm.state_dict(), path_lstm)

print(f"Run finished!")

# Log parameters

logging.info(f"hyperparams LSTM: {params_lstm}")
logging.info(f"LSTM {average_traj_err_train_lstm}")   
logging.info("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
logging.info("\n")

In [ ]:
# main (set parameters, logging, save model) 

params_tcn =    {
                        "window_size" : 30,
                        "learning_rate" : 0.001,
                        "batch_size" : 20,
                        "percentage_of_data" : 0.9,
                        "cut_off_timesteps" : 300,
                        "part_of_data" : 10,
                        "epochs" : 100,
                        "test_every_epochs" : 2,

                        "kernel_size" : 7,
                        "dropout" : 0,
                        "n_hidden" : 5,
                        "levels" : 4,
                        "input_channels" : 5,
                        "output" : 3
                    }

# Configure logging
log_file = 'training_fullsystem.log'
filemode = 'a' if os.path.exists(log_file) else 'w'
logging.basicConfig(filename=log_file, filemode=filemode, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize the TCN model
input_channels = params_tcn["input_channels"]
output = params_tcn["output"]
num_channels = [params_tcn["n_hidden"]] * params_tcn["levels"]
kernel_size = params_tcn["kernel_size"]
dropout = params_tcn["dropout"]
model_tcn = OR_TCN(input_channels, output, num_channels, kernel_size=kernel_size, dropout=dropout, windowsize=params_tcn["window_size"]).to(device)

# Generate input data (the data is normalized and some timesteps are cut off)
input_data1 = get_data(path = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_100.csv", num_inits=params_tcn["part_of_data"])
input_data2 = get_data(path = r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_randomwalks.csv", num_inits=params_tcn["part_of_data"])

input_data = torch.cat((input_data1, input_data2))
print(input_data.size())

#Split data into train and test sets
np.random.seed(1234)
num_of_inits_train = int(len(input_data)*params_tcn["percentage_of_data"])
train_inits = np.random.choice(np.arange(len(input_data)),num_of_inits_train,replace=False)
test_inits = np.array([x for x in range(len(input_data)) if x not in train_inits])
np.random.shuffle(train_inits)
np.random.shuffle(test_inits)
train_data = input_data[train_inits,:input_data.size(dim=1)-params_tcn["cut_off_timesteps"],:]
test_data = input_data[test_inits,:,:]

# dataloader for batching during training
train_set_tcn = custom_simple_dataset(train_data, window_size=params_tcn["window_size"])
train_loader_tcn = DataLoader(train_set_tcn, batch_size=params_tcn["batch_size"], pin_memory=True)

average_traj_err_train_tcn = []

#Training loop
for e in tqdm(range(params_tcn["epochs"])):
    
    train_error = train_tcn(train_loader_tcn, model_tcn, learning_rate= params_tcn["learning_rate"])
    if e % 10:
        print("Training error : ", train_error)

    # Every few epochs get the error MSE of the true data
    # compared to the network prediction starting from some initial conditions
    if (e+1)%params_tcn["test_every_epochs"] == 0:
        err_test_tcn = test(test_data.to(device), model_tcn, model_type="tcn", window_size=params_tcn["window_size"], display_plots=True, numb_of_inits = 1)
        average_traj_err_train_tcn.append(err_test_tcn)
        print(f"Average error over full trajectories: training data TCN: {err_test_tcn}")
        
# Save trained model
path_tcn = f'Full_System/or_test_tcn.pth'

torch.save(model_tcn.state_dict(), path_tcn)

print(f"Run finished!")

# Log parameters

logging.info(f"hyperparams tcn: {params_tcn}")
logging.info(f"TCN {average_traj_err_train_tcn}")   
logging.info("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
logging.info("\n")


In [14]:

#main (set parameters, logging, save model) 

params_tcn =    {
                        "window_size" : 30,
                        "learning_rate" : 0.001,
                        "batch_size" : 20,
                        "percentage_of_data" : 0.9,
                        "cut_off_timesteps" : 300,
                        "part_of_data" : 0,
                        "epochs" : 100,
                        "test_every_epochs" : 2,


                        "kernel_size" : 7,
                        "dropout" : 0,
                        "n_hidden" : 8,
                        "levels" : 6,
                        "input_channels" : 5,
                        "output" : 3,
                    }

params_lstm =    {
                        "window_size" : 16,
                        "h_size" : 8,
                        "l_num" : 3,
                        "learning_rate" : 0.001,
                        "batch_size" : 20,

                        "percentage_of_data" : 0.8,
                        "cut_off_timesteps" : 0,
                        "part_of_data" : 0,
                        "epochs" : 1000,
                        "test_every_epochs" : 100,

                        "T_max" : 500,

                        "experiment_number" : np.random.randint(0,1000)
                        }
params_lstm2 =    {
                        "window_size" : 25,
                        "h_size" : 12,
                        "l_num" : 3,
                        "learning_rate" : 0.001,
                        "batch_size" : 20,

                        "percentage_of_data" : 0.8,
                        "cut_off_timesteps" : 0,
                        "part_of_data" : 0,
                        "epochs" : 1000,
                        "test_every_epochs" : 100,

                        "T_max" : 500,

                        "experiment_number" : np.random.randint(0,1000)
                        }




# Configure logging
log_file = 'training_fullsystem.log'
filemode = 'a' if os.path.exists(log_file) else 'w'
logging.basicConfig(filename=log_file, filemode=filemode, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize the TCN model
input_channels = params_tcn["input_channels"]
output = params_tcn["output"]
num_channels = [params_tcn["n_hidden"]] * params_tcn["levels"]
kernel_size = params_tcn["kernel_size"]
dropout = params_tcn["dropout"]
model_tcn = OR_TCN(input_channels, output, num_channels, kernel_size=kernel_size, dropout=dropout, windowsize=params_tcn["window_size"]).to(device)

# Generate input data (the data is normalized and some timesteps are cut off)

input_data = load_data(params_lstm["part_of_data"])
print(input_data.size())

#Split data into train and test sets
np.random.seed(1234)
num_of_inits_train = int(len(input_data)*params_tcn["percentage_of_data"])
train_inits = np.random.choice(np.arange(len(input_data)),num_of_inits_train,replace=False)
test_inits = np.array([x for x in range(len(input_data)) if x not in train_inits])
np.random.shuffle(train_inits)
np.random.shuffle(test_inits)
train_data = input_data[train_inits,:input_data.size(dim=1)-params_tcn["cut_off_timesteps"],:]
test_data = input_data[test_inits,:,:]

# dataloader for batching during training
train_set_tcn = custom_simple_dataset(train_data, window_size=params_lstm["window_size"])
train_loader_tcn = DataLoader(train_set_tcn, batch_size=params_lstm["batch_size"], pin_memory=True)

model_tcn = OR_TCN(input_channels, output, num_channels, kernel_size=7, dropout=dropout, windowsize=params_tcn["window_size"]).to(device)
model_lstm = OR_LSTM(input_size=5, hidden_size=params_lstm["h_size"], out_size=3, layers=params_lstm["l_num"], window_size=params_lstm["window_size"]).to(device)
model_lstm1 = OR_LSTM(input_size=5, hidden_size=params_lstm["h_size"], out_size=3, layers=params_lstm["l_num"], window_size=params_lstm["window_size"]).to(device)
model_lstm2 = OR_LSTM(input_size=5, hidden_size=params_lstm2["h_size"], out_size=3, layers=params_lstm2["l_num"], window_size=params_lstm2["window_size"]).to(device)

path_lstm=r"C:\Users\strasserp\Documents\ventil_lstm\Full_System\or_lstm_0_36.pth"
path_lstm1=r"C:\Users\strasserp\Documents\ventil_lstm\Full_System\or_lstm_0_755.pth" #best !!!
path_lstm2=r"C:\Users\strasserp\Documents\ventil_lstm\Full_System\or_lstm_1_987.pth"

# path_lstm=r"/home/rdpusr/Documents/ventil_lstm/Full_System/or_lstm_0_598.pth"
# path_tcn =r"/home/rdpusr/Documents/ventil_lstm/Full_System/or_tcn496.pth"
model_lstm.load_state_dict(torch.load(path_lstm, map_location=torch.device(device)))
model_lstm1.load_state_dict(torch.load(path_lstm1, map_location=torch.device(device)))
model_lstm2.load_state_dict(torch.load(path_lstm2, map_location=torch.device(device)))

test_data=get_data(r"C:\Users\strasserp\Documents\ventil_lstm\data_fs\training_data_full_system_long_test.csv", 0)
test(test_data.to(device), model_lstm1, model_type="lstm", window_size=params_lstm["window_size"], display_plots=True, numb_of_inits = 1, fix_random=False, rescale=True)
#test(test_data.to(device), model_lstm1, model_type="lstm", window_size=params_tcn["window_size"], display_plots=True, numb_of_inits = 1, fix_random=True)

torch.Size([980, 494, 5])


0.0008114278773314172

In [10]:
%matplotlib qt

In [12]:
import torch
from get_data_fs import *
from NN_classes_fs import *
from dataloader_fs import *
from test_func_fs import *

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def plot_results(x, pred, rescale=False, window_size=1):
    
    SMALL_SIZE = 15
    MEDIUM_SIZE = 25
    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=15)    # legend fontsiz

    params = {'axes.titlesize': 20}
    plt.rcParams.update(params)
    
    if rescale:
        x = normalize_invert(x)
        pred = normalize_invert(pred)

    if x.dim() == 3:
        x = x.view(x.size(dim=1), x.size(dim=2))
    if pred.dim() == 3:
        pred = pred.view(pred.size(dim=1), pred.size(dim=2))



    figure , axs = plt.subplots(5,1, figsize=(9,9))
    figure.tight_layout(pad=1.0)

    stepsize = 2 * 2e-5 
    time = np.linspace(0,x.size(dim=0)* stepsize, x.size(dim=0))

    axs[0].plot(time, x.detach().cpu().numpy()[:, 0], color="darkgreen", label="$u_{NC}$")
    #axs[0].set_title("NC : Input Voltage 1")
    axs[0].set_ylabel("voltage in V")

    axs[1].plot(time, x.detach().cpu().numpy()[:, 1], color="darkgreen", label="$u_{NO}$")
    #axs[1].set_title("NO : Input Voltage 2")
    axs[1].set_ylabel("voltage in V")

    axs[2].plot(time, pred.detach().cpu().numpy()[:, 2], color="red", label="neural network")
    axs[2].plot(time, x.detach().cpu().numpy()[:, 2], color="blue", label="data", linestyle="dashed")
    #axs[2].set_title("pressure")
    axs[2].set_ylabel("pressure in bar")

    axs[3].plot(time, pred.detach().cpu().numpy()[:, 3], color="red", label="neural network")
    axs[3].plot(time, x.detach().cpu().numpy()[:, 3], color="blue", label="data", linestyle="dashed")
    #axs[3].set_title("position")
    axs[3].set_ylabel("position in m")
    #axs[3].set_xlabel(f"time [s]")

    axs[2].axvline(x=time[window_size], color='black', linestyle="dotted", label='start of prediction')
    axs[3].axvline(x=time[window_size], color='black', linestyle="dotted", label='start of prediction')

    axs[4].plot(time, pred.detach().cpu().numpy()[:, 4], color="red", label="neural network")
    axs[4].plot(time, x.detach().cpu().numpy()[:, 4], color="blue", label="data", linestyle="dashed")
   # axs[4].set_title("velocity")
    axs[4].set_ylabel("[m/s]")
    axs[4].set_xlabel(f"time [s]")

    axs[2].axvline(x=time[window_size], color='black', linestyle='dotted', label='start of prediction')
    axs[3].axvline(x=time[window_size], color='black', linestyle='dotted', label='start of prediction')
    axs[4].axvline(x=time[window_size], color='black', linestyle='dotted', label='start of prediction')

    if rescale:
        u1_max = 200  #Spannung in [V]              ... [0, 200]
        u1_min = 0
        u2_max = 200
        u2_min = 0
        p_max = 3.5*1e5 #Druck in [bar]             ... [1, 3.5]
        p_min = 1.0*1e5 #Umgebungsdruck in [bar]
        s_max = 0.605*1e-3     #Position [m]          ... [0, 0.0006]
        s_min = 0.0
        v_max = 0.6     #Geschwindigkeit in [m/s]   ... [-1.7, 1.7]
        v_min = -0.6    #Geschwindigkeit in [m/s]   ... [-1.7, 1.7]   
        axs[0].set_ylim(u1_min-10, u1_max+10)
        axs[1].set_ylim(u2_min-10, u2_max+10)
        axs[2].set_ylim(p_min-0.1*p_max, p_max+0.1*p_max)
        axs[3].set_ylim(s_min-+0.1*s_max, s_max+0.1*s_max)
        axs[4].set_ylim(v_min, v_max)

    for i in range(5):
        axs[i].grid(True)
        axs[i].legend()



    plt.grid(True)
    plt.legend()
    plt.show()

def test(data, model, model_type="tcn", window_size=1 ,display_plots=False, numb_of_inits=1, fix_random=True, rescale=False):

    if fix_random:
     np.random.seed(1234)
    else:
     np.random.seed(seed=None)
      
    test_inits = data.size(dim=0)
    ids = np.random.choice(test_inits, min([numb_of_inits, test_inits]), replace=False)
    ids = np.unique(ids)

    data = data[ids,:, :]

    loss_fn = nn.MSELoss()
    timesteps = data.size(dim=1)

    total_loss = 0
    for i, x in enumerate(data):

       
        with torch.inference_mode():
            if model_type == "tcn":
                
                x=x.to(device)        
                x = x.view(1,x.size(dim=0), x.size(dim=1))                
                pred = torch.zeros((timesteps, 5), device=device)

                pred[0:window_size, :] = x[0, 0:window_size, :]
                pred[:, 0] = x[0, :, 0]

                x_test = x.clone()
                x_test[:,window_size:,2:] = 0
                x_test = x_test.to(device)
                #print("Data passed to the model, all 0 after the initial window to prove that the forward pass is correct and doesnt access information it shouldnt.",x_test[:,0:10,:])

                out = model(x_test.transpose(1,2))
                
                pred[window_size:,2:] = out.squeeze(0).transpose(0,1)

                total_loss += loss_fn(pred[window_size:, 2:], x[0, window_size:, 2:]).detach().cpu().numpy()

                if display_plots:
                    plot_results(x, pred, rescale=rescale, window_size=window_size)

            if model_type == "lstm":
                x=x.to(device)        
                x = x.view(1,x.size(dim=0), x.size(dim=1))                
                pred = torch.zeros((timesteps, 5), device=device)

                pred[0:window_size, :] = x[0, 0:window_size, :]
                pred[:, 0] = x[0, :, 0]

                x_test = x.clone()
                x_test[:,window_size:,2:] = 0
                x_test = x_test.to(device)

                out, _ = model(x_test) 
                pred[window_size:,2:] = out

                total_loss += loss_fn(pred[window_size:, 2:], x[0, window_size:, 2:]).detach().cpu().numpy()

                if display_plots:
                    plot_results(x, pred, rescale=rescale, window_size=window_size)

            if model_type == "mlp":
                x=x.to(device)        
                x = x.view(1,x.size(dim=0), x.size(dim=1))                
                pred = torch.zeros((timesteps, 5), device=device)

                pred[0:window_size, :] = x[0, 0:window_size, :]
                pred[:, 0] = x[0, :, 0]
    
                x_test = x.clone()
                x_test[:,window_size:,2:] = 0
                x_test = x_test.to(device)

                out = model(x_test)
                pred[window_size:,2:] = out

                total_loss += loss_fn(pred[window_size:, 2:], x[0, window_size:, 2:]).detach().cpu().numpy()

                if display_plots:
                    plot_results(x, pred, rescale=rescale, window_size=window_size)

    return total_loss/data.size(0)